In [20]:
# coding=utf-8

import pymysql
import os
import math
import requests
import pandas as pd
import jieba
import re
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score, calinski_harabaz_score
from sqlalchemy import create_engine
from sklearn.cluster import KMeans

def cut_word(words):   #分词
    result = jieba.cut(words,cut_all=False)  #使用精确模式
    new_words = []
    for r in result:
        new_words.append(r)
    return new_words

def del_stop_words(words,stop_words_set):
#   words是已经切词但是没有去除停用词的文档。
#   返回的会是去除停用词后的文档
    new_words = []
    for r in words:
        if r not in stop_words_set:
            new_words.append(r)
    return new_words

f1 = open('C:/Users/win8/Desktop/1111stop.txt','r', encoding='utf-8')
stopword=f1.read()

str1 = input("请输入文章内容：")
print("你输入的内容是: ", str1)

db = pymysql.connect(host="127.0.0.1",database="weixin",user="root",password="yt6655050",port=3306,charset='utf8mb4')   #链接数据库
with db:
    cur = db.cursor()
    # cur.execute("truncate table weixin.train")
    # cur.execute("SELECT * FROM after_cut WHERE 公众号名称 LIKE '牛气电商'")
    # cur.execute("SELECT * FROM after_cut  WHERE 公众号类型 LIKE '零售'")
    cur.execute("SELECT * FROM train")
    numrows = int(cur.rowcount)
    print(numrows)
    rows = cur.fetchall()
    #写入训练集数据
    f = open('C:/Users/win8/PycharmProjects/textmining/test.txt', 'w',encoding='utf-8')
    for i in range(numrows):
        r=''.join(rows[i][17].split('\r\n'))
        f.write(rows[i][9]+'\t'+r)
        f.write("\n")#换行
    r1 = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>"关闭""微信""赞赏""二维码""公众""号""苹果公司""转账""赞赏""别忘了""支持""猜""喜欢""评论""留言""影响""版""功能"?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'#用户也可以在此进行自定义过滤字符
    r2 = u'\s+;'
    tem1 = re.sub(r1, '', str1)  # 过滤文本中的所有英文数字和符号
    tem1 = cut_word(tem1)
    tem1 = del_stop_words(tem1, stopword)
    tem2 = ','.join(tem1)
    text_content=str(tem2)
    fans_num = 547468  # 输入公众号粉丝数
    ddd = ''.join(text_content.split('\r\n'))
    f.write("未分类" + '\t' + ddd)
    f.write("\n")

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/test.txt', sep='\t', header=None, )
print(train_df.shape)
train_df = train_df.astype(str)
print(train_df.shape)

train_df.columns = ['分类', '文章']
stopword_list = [k.strip() for k in open('C:/Users/win8/PycharmProjects/textmining/venv/1111stop.txt', encoding='utf8').readlines() if k.strip() != '']
cutWords_list = []
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' %(i, time.time()-startTime))
    cutWords_list.append(cutWords)
print(i)
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt', 'w',encoding='utf-8') as file:
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt',encoding='utf-8') as file:
    cutWords_list = [k.split() for k in file.readlines()]

请输入文章内容：又一个手机勒索木马，都说别乱下软件了……    原创：  谢幺    浅黑科技   浅黑科技   微信号 qianheikeji   功能介绍 在未来面前，我们都是孩子。            “网络勒索是最完美的商业模式”，我不止一次听到这样的说法，事实也几乎如此。WannaCry勒索病毒风波过去已近半年，凶手抓到没？一个在全球掀起轩然大波，明目张胆抢劫的大案，就这么平息了，凶手不知还在哪个海岛度假。一波未平一波又起，电脑勒索大案的真凶还未归案，手机勒索也开始了……?10月13日ESET研究人员发现一个手机勒索软件，不仅将用户手机里的资料加密，还能修改手机解锁图案，让你无法正常使用。?倒计时、支付额、加密文件数，可以看出，界面内容同PC电脑勒索软件无异，勒索效果也同样简单粗暴。ESET安全研究人员将这种新型手机勒索木马命名为Doublelocker（双重锁），因为它不仅会加密用户手机里的资料文件，还会修过屏幕解锁图案或PIN码，防止受害者打开手机。这是首次发现手机勒索软件滥用安卓系统自带“辅助功能”的案例。?“辅助功能”是安卓系统自带的特殊功能，开启后可以授权应用进行一些特殊便利性操作，但同时也需要对软件进行特殊授权。?以搜狗输入法为例，它拥有一项“智能回复”功能，号称收到对方的消息后能自动回复对应消息，不需要打字，但代价就是授权程序监控你的手机画面，用于观看和分析对方发来的消息内容。该功能在“辅助功能”界面▼Doublelocker勒索软件利用了辅助功能可获取特殊权限的特点，将木马安装包伪装成AdobeFlashPlayer播放器，引导用户下载并开启辅助功能。?浅黑科技在该程序辅助功能授权界面发现，该程序要求的几样特殊权限：擦除所有数据更改锁屏密码设置密码规则一旦用户不注意该页面的权限，直接点击确定，手机就会被锁定。虽然这种手法看起来非常拙劣，但如今不少手机用户在安装使用软件时并不会认真查看软件权限，给了恶意木马可乘之机。勒索木马被授权后，会修改设备PIN码，让受害者无法使用手机，只有用户支付赎金才能远程解锁或重置密码。?然后，DoubleLocker会将设备主要存储目录加密成?cryeye后缀的文件，据有关专家介绍，由于勒索软件使用了高强度的AES加密算法，因此在没有密钥的情况下几乎无法恢复文件。一旦中招怎么办？相关安全人员表示，手机PIN码被

In [21]:
tfidf = TfidfVectorizer(cutWords_list, min_df=10, max_df=0.6)
X = tfidf.fit_transform(train_df['文章'])
print(X.shape)

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/test.txt', sep='\t', header=None)
train_df = train_df.astype(str)
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df[0].values)

#降维
from sklearn.feature_selection import SelectKBest,chi2
X=SelectKBest(chi2,k=500).fit_transform(X,y)

X1=X[2006]

#划分训练测试集
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=9,min_samples_split=3)
model.fit(train_x, train_y)

(2007, 11580)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:

predict = model.predict(X1)

for i in range(len(y)):
    if predict == y[i]:
        print(train_df[0].values[i])
        corpus = []
        db = pymysql.connect(host="127.0.0.1", database="weixin", user="root", password="yt6655050", port=3306,
                             charset='utf8mb4')  # 链接数据库
        with db:
            cur = db.cursor()
            leixing = str(train_df[0].values[i])
            sqlword = "SELECT * FROM train WHERE 公众号类型 LIKE " + "'" + leixing + "'"
            cur.execute("truncate table weixin.after_vector")
            cur.execute(sqlword)
            numrows = int(cur.rowcount)
            print(numrows)
            rows = cur.fetchall()
            for i in range(numrows):
                corpus.append(rows[i][17])  # 存放所有文档分词结果
                insert_data = ("INSERT INTO after_vector value(" + str(i) + ",'" + str(rows[i][0]) + "')")  # 存放索引
                try:
                    cur.execute(insert_data)
                except:
                    db = pymysql.connect(host="127.0.0.1", database="weixin", user="root", password="yt6655050",
                                         port=3306,
                                         charset='utf8mb4')  # 链接数据库
                    cur = db.cursor()
                    cur.execute(insert_data)
                db.commit()
            #插入预测目标
            corpus.append(text_content)
            i=numrows
            id="1234567"
            insert_data = ("INSERT INTO after_vector value(" + str(i) + ",'" + str(id) + "')")  # 存放索引
            try:
                cur.execute(insert_data)
            except:
                db = pymysql.connect(host="127.0.0.1", database="weixin", user="root", password="yt6655050",
                                     port=3306,
                                     charset='utf8mb4')  # 链接数据库
                cur = db.cursor()
                cur.execute(insert_data)
            db.commit()

        vectorizer = CountVectorizer(max_df=0.6, min_df=3)  # 将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
        X = vectorizer.fit_transform(corpus)
        word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
        # print(word)
        print(word.__len__())  # 词典的长度
        # print(word.__getitem__(1))#获得词典中特定位置的词
        pd.DataFrame(X.toarray(), columns=word)
        # print(pd.DataFrame(X.toarray(), columns=word)) #输出词频矩阵
        transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值,设置当设置为浮点数时，过滤出现在超过max_df/低于min_df比例的句子中的词语；正整数时,则是超过max_df句句子。这样就可以帮助我们过滤掉出现太多的无意义词语。
        tfidf = transformer.fit_transform(
            vectorizer.fit_transform(corpus))  # 第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
        weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
        # print(weight.__len__())
        weight_data = pd.DataFrame(weight, columns=word)

        print('Start Kmeans:')

        clf = KMeans(n_clusters=10, random_state=1, algorithm='auto', n_init=10, init='k-means++')
        s1 = clf.fit(weight)
        s = clf.fit_predict(weight)

        j = 252  # 选取需要输出的簇
        print(clf.labels_[j])
        # 将簇内的文章输出
        for i in range(len(clf.labels_)-1):
            if clf.labels_[i] == clf.labels_[j]:
                print(i)
                sqlword = "SELECT * FROM after_vector WHERE 列数 LIKE " + "'" + str(i) + "'"
                cur.execute(sqlword)
                rows = cur.fetchall()
                k = rows[0][1]
                sqlword = "SELECT * FROM train WHERE id LIKE " + "'" + str(k) + "'"
                cur.execute(sqlword)
                rows = cur.fetchall()
                print("公众号名称：",rows[0][2])
                print("文章名称：",rows[0][1])
                print("文章评分：",rows[0][15])
                print("发文日期:",rows[0][6])
                print("发文时间:",rows[0][7])
                print("图片数:",rows[0][8])
                print("周发文数:",rows[0][12])
                print("文章内容:",rows[0][17])
        db.close()
        break

互联网
252
6859
Start Kmeans:
2
4
公众号名称： 浅黑科技
文章名称： 又一个手机勒索木马,都说别乱下软件了
文章评分： 39.801
发文日期: 2017-10-18
发文时间: 15
图片数: 10
周发文数: 2
文章内容: 手机,勒索,木马,乱,软件,原创,谢幺,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,网络,勒索,完美,商业模式,听到,说法,事实,勒索,病毒,风波,近半年,凶手,抓到,全球,掀起,轩然大波,明目张胆,抢劫,大案,平息,凶手,海岛,度假,一波未平,一波又起,电脑,勒索,大案,真凶,归案,手机,勒索, ,研究,人员,发现,手机,勒索,软件,用户,手机,资料,加密,修改,手机,解锁,图案, ,倒计时,付额,加密,文件,界面,内容,电脑,勒索,软件,无异,勒索,简单,粗暴,研究,人员,新型,手机,勒索,木马,命名,双重,锁,加密,用户,手机,资料,文件,还会,修过,屏幕,解锁,图案,受害者,手机,这是,首次,发现,手机,勒索,软件,滥用,安卓,系统,自带,辅助,案例, ,辅助,安卓,系统,自带,开启,授权,便利性,操作,软件,授权, ,搜狗,输入法,为例,拥有,一项,智,回复,收到,消息,自动,回复,消息,打字,代价,授权,程序,监控,手机,画面,用于,观看,分析,发来,消息,内容,辅助,界面,▼,勒索,软件,利用,辅助,获取,权限,木马,安装包,伪装成,播放器,引导,用户,下载,开启,辅助, ,浅,黑,科技,程序,辅助,授权,界面,发现,程序,几样,权限,擦除,数据,更改,锁,屏密,设置,规则,用户,页面,权限,点击,手机,锁定,手法,拙劣,手机用户,安装,软件,查看,软件,权限,恶意,木马,可乘之机,勒索,木马,授权,修改,设备,受害者,手机,用户,付,赎金,远程,解锁,重置, ,设备,存储,目录,加密, ,后缀,文件,专家,介绍,勒索,软件,高强度,加密算法,密钥,情况,恢复,文件,中招,人员,手机,改,好办,手机,连接,电脑,工程模式,清除,系统,数据文件,即可,清除,屏幕,锁屏,图案,模式,卸载,恶意软件, ,加密,数据文件,恢复,付,赎金,手机,勒索,软件,电脑,勒索,软件,本质,麻烦,研究者,奉劝,非正规,渠道,下载安装,软件, ,网络,勒索,一本万利,商业模式,现今,遭受,严厉打击,区块,链,数字

公众号名称： 一个程序员的日常
文章名称： 再也不敢点外卖了
文章评分： 386.064
发文日期: 2019-03-25
发文时间: 09
图片数: 3
周发文数: 6
文章内容: 再也,外卖,原创,路人甲,程序员,日常,程序员,日常,介绍,程序员,方式,世界,作者,名字,终于,回归,路人甲,名字,时间,互联网,忠实,用户,隐私,暴露,事情,吓怕,身份证,卖,盗用,事儿,发现自己,地理位置,人想,小时,吓怕,变得,警惕,变得,小心翼翼,遗憾,学会,明察秋毫,是因为,隐私,这件,事情,件,好事儿,明察秋毫,之下,资深,用户,发现,朋友,想,吃,后脚,外卖,发现,外卖,推荐,这家,想,买,东西,后脚,淘宝,推荐,商品,更有甚者,想,买,本书,淘宝,推荐,那本书,妹子,男朋友,分手,两个,淘宝,推荐,自卫器,男朋友,晚上,淘宝,偷听,细思极,类似,这件,事儿,搜,搜榜待,发现,资深,用户,晓之以理,动之以情,推断出,美团,饿,一类,外卖,软件,淘宝,一类,购物,软件,小时,监听,手机,发出,声音,分析,推荐,接受,反驳,推断,技术,角度,理解,两句,技术,这件,事情,可行,技术,麦克风,监控,理解,可行,下载,美团,外卖,一类,软件,访问,麦克风,权限,麦克风,权限,监控,系统,摆设,系统,摆设,系统,真的,监控,手机,话筒,一家,监控,手机,查,中国,亿多,用户,美团,外卖,活饿,外卖,月活,月活,注册,用户,肯定,亿,监控,手机,外卖,软件,累不累,肯定,有人,监控,活跃,用户,闲,无聊,监控,监控,话筒,不行,语音,东西,电脑,平台,语音,准确,文字,文字,提取,推荐,语音,事情,复杂度,判断,一件,事情,难度,高不高,这件,事情,单独,产业,语音,技术,慢慢,兴起,产业,讯,飞,百度,阿里,挤破,脑袋,搞,世界,一流,科技,发展,语音,识到,最最,最牛,逼,系统,准确率,实验,条件,英语,中文,商用,语音,系统,准确率,估计,肯定,低于,普通话,情况,手机,讲话,才识,普通话,夹杂着,手机,放在,口袋,情况,准确率,有没有,一套,语音,中文,分析,用户,十种,食物,吃,东西,分析,有人,监听,键词,监听,键词,用户,一句,「,讨厌,寿,推荐,寿,实际上,一道,题目,技术,很难,判断,一套,流程,真的,做好,估计,世界,第一,市值,企业,外卖,估计,有人,推翻,那

公众号名称： 浅黑科技
文章名称： 揭黑,有人在微信里公然开赌场,月入过亿,代理商遍布全国,有人一夜输9万,却为何无人报警查处
文章评分： 325.678
发文日期: 2018-04-11
发文时间: 10
图片数: 38
周发文数: 2
文章内容: 揭黑,有人,然开,赌场,月入,亿,代理商,遍布全国,有人,一夜,输万,无人,报警,查处,原创,浅,黑,科技,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,罪恶,可怕,所有人,浑然不觉, , ,受害者,前仆后继,监管者,视而不见,开路,放行,事儿,真实,发生,浅,黑,科技,接到,读者,爆料,有人,利用,推广,新型,网络,赌博,项目,受害者,遍布全国,有人,一夜,输掉,十万元,可怕,项目,精心,包装,成网购,商城,披上,娱乐,购物,外衣,受害者,骗,商城,自认,教唆,赌博,项目,火爆,招商,堂而皇之,群,朋友圈,拥有,数十个,然在博,百度,推广,招商,商店,腾讯,宝,上架,明星,代把,广告,挂到,权威,新闻,网站,起底,平台,商店,明目张胆,新型,网络,赌博,文浅,黑,科技,赌场,光鲜,外衣,报警,家住,湖南,农村,杨,一晚,输掉,积蓄,已久,万块,家人,劝,报警,情愿,报警,警察,买,万多块,钱,茶叶,不报,事实上,老杨,不服,想,借点,钱,回本,几个,网友,杨,推荐,加斯,商城,声称,买,东西,赚大钱,好几百,月入,万元,梦,投得,赚,单车,摩托,老杨,心想,商城,买,东西,花钱,商城,买,东西,赚钱,好奇,他点,官方,宣传,这家,商城,声称,国内,最新,娱乐,购物,模式,顾客,买,东西,玩游戏,赚钱,平常,商城,无异,杨点,商城,看似,平常,网购,页面,促销,专区,网友,告诉,加斯,商城,娱乐,购物,核心,赚钱,键,促销,专区,页面,玩法,顾客,商城,买,一件,商品,促销,机会,促销,抽奖,成就,价格,买到,一件,价值,商品,退货,钱,促销,失败,顾客,依然,买,商品,退货,买,东西,免费送,抽奖,中奖,赚钱,输,买,等值,商品,听,稳赚,不亏,生意,网友,鼓动,杨,买个,茶叶,试试,购买,商品,老杨,促销,页面,促销,活动,分为,两种,拆,红包,鸡,藕,拆,红包,简单,粗暴,页面,两个,红包,点开,告诉,促销,类似,熟知,砸,金蛋,相比之下,鸡,藕,有意思,专业,鸡,藕,奇偶,单数,双数,开奖,正规,福利彩票

公众号名称： 浅黑科技
文章名称： 重返聚光灯,绿盟做对了什么
文章评分： 147.698
发文日期: 2018-04-16
发文时间: 12
图片数: 15
周发文数: 2
文章内容: 重返,聚光灯,绿盟,原创,史中,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,浅友们,史,日常生活,撩,五湖四海,科技,大牛,我会,尝试,姿势,无边,脑洞,温情,故事,听,特想,听到,故事,告诉,反正,撩,重返,聚光灯,绿盟,文史,中浅,黑,科技,眼里,聚集,中国,顶尖,黑客,绿盟,沉默,想到,脑海中,浮现,透明,绿巨人,坐在,桌边,喃喃自语,讲道理,传奇,过往,显得,搭配,绿盟,传奇,往事,中哥,开贴,聊有,想,听,童鞋,举个,爪,世界,善良,之处,猝不及防,温情,反年,元旦,绿盟,股价,结束,两年,下跌,不到,块,最低点,回升,声音,嘈杂,舆,背景,声中,变得,清晰,高管,出街,发布,行业,报告,密度,提高,感一波,高过,一波,闷头,赚钱,闷头,赚钱,老哥,重返,聚光灯,五个,字,送给,绿盟,毛病,归来,历程,绿盟,冷暖自知,旁观者,角度看,我心,两个,否从,绿盟,身上,学到,传统,企业型,标准,姿势,个体,否从,绿盟,身中,经验,沉默,许久,重返,战场,绿盟,操作,三招,希望,第一招,精选,一首,主打,歌,心中,绿盟,产品,见长,擅长,工具,产品,刀枪剑戟,十八般,兵刃,挂满,墙,眩晕,电视剧,片尾曲,歌手,唱,挑大拇哥,不温不火,原因,主打,歌,形象,钉,观,脑子里,绿盟,返回,战场,迫切需要,主打,歌,主打,歌会,绿盟,诸多,硬件,执着,工具,并不一定,带来,生存,优势,想到,栗子,解放战争,时期,国民党,军,配置,重型,美式,装备,解放军,日本,缴获,轻型,武器,标准,美式,装备,两个,弱点,道路,平整度,耗费,源,小米,步枪,中国,一穷白,破破烂烂,现实情况,机动性,爆棚,战争,人尽皆知,解放军,胜利,原因,战争,终点,消灭,对手,先进,武器,战争,赛博,世界,翻,网络安全,企业,用户,想要,终点,解决,拥有,产品,终点,反推,绿盟,找到,主打,歌,威胁,情报,打个,硬件,四肢,胸肌,腹肌,威胁,情报,认知,大脑,智商,绿盟,硬件,最强,优势,这是,路人甲,事实,想,强壮,身躯,意味着,打架,赢,看过,霍元甲,体会,感觉,更为重要,出拳,格斗,拳路,风格,认知,

公众号名称： 浅黑科技
文章名称： 无线电狂人,简云定,三秒干掉智能锁的骚操作是如何做到的
文章评分： 252.801
发文日期: 2018-07-01
发文时间: 10
图片数: 30
周发文数: 2
文章内容: 无线电,狂人,简云定,三秒,干掉,智锁,骚,操作,原创,史中,浅,黑,科技,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,浅友们,史,日常生活,撩,五湖四海,科技,大牛,我会,尝试,姿势,无边,脑洞,温情,故事,听,特想,听到,故事,告诉,无线电,狂人,简云定,揭秘,三秒,干掉,智锁,骚,操作,文史,中浅,黑,科技,三秒钟,电击,开智,门锁,骚,操作,火,操作,简单,黑盒,门锁,一晃,门锁,动手,我家,大门,新闻,脑海,工作,无线电,狂人,简云定,此人,狂人,夸张,诗,眼中,无线电,初恋,满屋,电子,元器件,不合,开焊,工作室,手下,机器,芯片,焊到,电路板,贴片机,干,工作,挣万,买,元器件,人神共愤,事情,简云定,取个,孤独,小白,疯魔,可注,孤生,同事,投来,怜悯,目光,殊不知,此人,孤独,两个,孩子,爹,回,无线电波,世界,太平,时不时,有人,搞,小盒,干蒙,小区,门锁,机隔,车窗,盗,刷卡,骚,操作,找,孤独,小白,聊聊,肯定,没错,风靡一时,用机,隔空,刷卡,晴朗,午后,找到,简云定,听说,黑盒,三秒,搞定,智,门锁,顺手,后腰,身体,中后,偏下,部位,掏出,一坨,黑乎乎,东西,体温,黑盒子,掏出,黑盒,疼,东西,简云定,黑盒,特斯拉,线圈,抖,音上,搜一,搜,键词,肯定,刷新,认知,东西,结构,简单,一块,电池,一块,升压,电路,发射,天线,零件,买,小白,教程,组装,很快,史中,简云定,口中,小白,也许,理解,小白,一种,小白,那种,手机,贴膜,歪,残党,谨慎,特斯拉,线圈,发出,电磁波,电磁炉,波炉,原理,做出,人造,闪电,电火花,音箱,各种各样,玩具,电磁,音箱,电火花,音乐舞蹈,为啥,智,门锁,简云定,黑盒,电压,挺高,十伏,把量,电磁波,形式,放出去,电磁波,门锁,电路,一把,智锁,主板,电机,控制电路,电机,机械锁,装置,主板,传给,电机,控制电路,控制电路,电机,发出,工作,电,机动,带动,机械锁,要说,锁,黑盒,两种,情况,电磁波,主板,干,死机,门锁,重启,逻辑设计,重启,大部分,锁簧,电磁波,锁,芯,电机,控制电路

公众号名称： 浅黑科技
文章名称： 扇自己脸,PhotoShop,的公司研发了一套,人工智能反P图,技术
文章评分： 176.859
发文日期: 2018-07-05
发文时间: 12
图片数: 49
周发文数: 2
文章内容: 扇,脸,研发,一套,人工,智反图,技术,原创,谢幺谢,幺,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,网上,冲浪者,必修,知识点,有人,有人,分不清,专业,网民,绝逼,听说,真假难辨,充斥,易容术,网络,世界,迷失,吴亦凡,当成,胖虎,张学友,当成,王力宏,心中,地位,之高,发合时,一度,心中,鉴定,真闺蜜,男友,铁哥们,标准,美图秀,秀,工具,习惯,图而,美,字,深深,烙印,心中,有家,研发,一项,「,技术,人工,智来识,图像,篡改, , ,精致,秒,秒钟,有意思,研发,这项,技术,「,字,钢印,烙,心中,爹, , ,王牌,软件,研发,人工,智反图,技术,脸,原理,检测, ,两个,疑惑,发现,一段, , ,技术,往事, , , ,这位,大叔,名叫,约翰,诺尔,鼎鼎大名,特效,「,工业,光魔,首席,创意,官, ,听,「,工业,光魔,至少,听过,皮克斯,工作室,皮克斯,前身,工业,光魔,动画,工作室,当年,乔布斯,花万买,成立,独立,这才,乔布斯,辉煌,故事,好莱坞,一多半,电,特效,出自,工业,光魔,熟知,星球大战,系列,加勒比海,盗阿凡,这位,约翰,大叔,更是,工业,光魔,荣获,十次,奥斯卡,最佳,视觉,特效,奖,飞起, ,伟大成就,缔造者,回到,张图,桌上,摆着,一台,早期,电脑,里头,用史,第一个,商业,展示,史,第一张,照片,一张,海滩,美女,照片,画面,约翰,大叔,老婆,时间,回到, ,那年,约翰,大叔,年轻,帅气,小伙子,工业,光魔,上班,忙,一部,大片,女朋友,詹妮弗,太平洋,中部,波拉,波拉,岛,休假,清澈,海滩,詹妮弗,拍下,这张,照片,求婚,詹妮弗,惊,意识,这张,背会,人类,科技史,浓墨重彩,一笔,画面,地球,另一侧,电脑前,密歇根,大学,计算机,视觉系,在读,博士,托马斯,正坑,呲,坑,呲,干活,并不知道,老哥,享受,阳光,沙滩,美女,未来,嫂子,作伴,将来,哥哥,一件,改变,世界,事情,埋头,研发,一套,名叫, , ,电脑,程序,显示,修改,数字图像,最早,原型,市面上,类似,软件,约翰,度假,回国,找,博

公众号名称： 浅黑科技
文章名称： AI,开发者,的热血时代
文章评分： 127.545
发文日期: 2018-07-10
发文时间: 12
图片数: 20
周发文数: 2
文章内容: 开发者,热血,时代,原创,史中,浅,黑,科技,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,浅友们,史,日常生活,撩,五湖四海,科技,大牛,我会,尝试,姿势,无边,脑洞,温情,故事,听,特想,听到,故事,告诉,开发者,热血,时代,文史,独立,宣中,生而平等,也许,是因为,努力,身边,平等,总有,遗憾,这篇,文字,童鞋,肯定,幸运,双眼,感知,世界,光辉,此时此刻,楼下,街道,盲道,空空荡荡,自行车,报刊亭,占领,沉默,争辩,记得,采访,结束,横穿,一条,三四十米,马路,走,隔离带,旁边,一位,手杖,老奶奶,方向,小心翼翼,地问,可不可以,看着,车,盲人,惊讶,回头,看刚,走过,半条,马路,车水马龙,一架,架,钢铁,躯壳,呼啸而过,走,难以想象,感觉,危险,情况,下才,尴尬,求助,扶,车站,告时,不让,家人,陪,老奶奶,孩子,很忙,麻烦,这句,深深地,记忆里,盲人,可不,室外,悠悠,蝉鸣,清新,空气,选择,躲,家里,冒险,出门,怕麻烦,家人,勾起,这段,回忆,一位,梁佳,大学生,梁佳,早些时候,利用,百度,人工,智,平台,开发,一套,盲人,导路,设备,设备,戴,头上,摄像头,自动,采集,眼前,环境,语音,震动,提示,盲人,障碍物,简单,精巧,设计,不可思议,大赛,冠军,前两天,百度,开发者,大会,李彦宏,演讲,临近,尾声,点名,感谢,梁佳,网络,直播,全球,观,李彦宏,百度,开发者,大会,梁佳,学生,善良,热爱,技术,骚年,称谓,开发者,四个,百度,开发者,故事,开发者,经历,中有,一句,证实,胜利,时代,胜利,两百年,梦想,马镫,锻造,师,优秀,铸剑师,十年,梦想,机系统,设计师,娴熟,胶片,冲印,师,站,回看,职业,骄傲,永远,时光,退场,遥想,年代,一代,股神,杰西,利弗莫,利用,内幕,交易,操纵,股价,狂,赚,数亿美元,晚年,证券,市场监管,失算,黯然,谢幕,所有人,熟悉,巴菲特,价值,投资,新一代,明星,职业,消亡,职业,到来,晚清,洋务运动,救亡图存,登高一呼,我要,开发者,人工,智,改进,汉阳铁,厂,钢铁,良品率,多半,送,就医,梁佳,骄傲地,宣称,一名,开发者,此

公众号名称： 浅黑科技
文章名称： 这么皮的酒店,我还是第一次住
文章评分： 414.085
发文日期: 2018-08-03
发文时间: 11
图片数: 25
周发文数: 2
文章内容: 皮,酒店,第一次,住,原创,史中,浅,黑,科技,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,中哥,慌,一笔,男人,邀,开房,刷开,房门,妹子,想,听,慢慢说,朋友,叫俊林,三个,身份,音乐,发烧友,伪,艺术家,兼云,计算,徒前,两个,特长,不靠,谱,导致,吃,土,腾讯,工作,俗话说,不想,搞,事情,白云,黑土,区,俊林,满脑子,想,用云,计算,改变,群,生活,包括,一段时间,天天,追着,腾讯,技术,我家,房子,改装成,语音,控制,马桶,语音,控制,浴缸,语音,控制,电视,灯光,语音,控制,坐在,沙发,沐浴,倾泻,温暖,灯光,空气,想,电,射灯,熄灭,床头灯,恰到好处,发出,质感,窗帘,识时务,投,电视,怦然,亮起,电,顺次,屏幕,安静,等待,召唤,这话,双眼,迷离,要施,魔法,仿佛,手里,哈利波,棍子,思路,一句,云端,智,控制,平台,控制,听,叼,炸天,听说,方案,改造,线路,改动,装修,感动,拒绝,狂人,俊林,贼心不死,春天,同事,开会,顿悟,智,房间,设计,酒店,适合,研究,发现,亚朵,酒店,火,一笔,冷不丁,推出,情趣,主题,酒店,吴晓波,酒店,壳,网房,同道,大叔,房,差房,差房,图片,差君,黑胖,广告费,图片,缩小,找到,亚朵,酒店,一套,词儿,迷离,双眼,一遍,亚朵,酒店,创意,叼,拿出,几个,样板间,腾讯,改造,目标,几天,房间,改造,男人,邀,酒店,体验,一幕,高潮迭起,分界线,俊林,一张,房卡,眼神,好好,体会,刚刷开,房门,屋子里,传出,温柔,女声,名字,纳尼,正经,亚朵,酒店,那种,羞羞,服务,联想,俊林,眼神,不做不休,小姐姐,我来,进屋,失望,声音,是从,发出,声音,桌子,智,音箱,猛地,抬头,发现自己,名字,屏幕,看不清,一张,高清,史,体会,这套,皮,入住,体验,俊林,走进,眯起,双眼,表演,吼,一声,嗨,电,屋里,灯光,识趣,地暗,窗帘,羞答答,地合,投,电视,调整,腾讯,视频,描述,一毛,气氛,怪怪的,卧槽,一圈,地灯俊林,地灯,自动,感应,地下,站,自动,熄灭,躺,床上,试试看,它会,灭,严词拒绝,目光,聚焦,电视,发现,腾讯,视频,

公众号名称： 浅黑科技
文章名称： 我去拜访了,专门鉴定吻戏的老湿傅
文章评分： 243.702
发文日期: 2018-08-19
发文时间: 10
图片数: 38
周发文数: 2
文章内容: 拜访,鉴定,吻,戏,老湿傅,原创,史中,浅,黑,科技,浅,黑,科技,浅,黑,科技,介绍,未来,面前,孩子,拜访,鉴定,吻,戏,老湿傅,文史,前两天,中哥,手机,消息,揭秘,吻,戏,鉴定,师,日常,操作,接吻,中哥,一下子,椅子,上勃,纳尼,羞羞,操作,这位,眼神,坚毅,中带,柔情,老湿傅,可要,火有,这位,吻,戏,鉴定,师,新闻报道,冷静,仔细,新闻,老湿傅,优酷,一位,算法,工程师,日常,工作,用极,骚极骚,算法,一秒,找到,特定,人物,情节,吻,戏,戏,编辑,剪辑,预告片,集锦,天底下,福利,工作,五湖四海,寻找,技术,大牛,中哥,老湿傅,这位,朋友,我交定,手纸,易容,飞檐走壁,摸,阿里巴巴,北京,办室,摸,这位,老湿傅,脸颊,经历,全程,分割线,中哥,优酷,奇遇记,老湿傅,名叫,吉恒杉,听到,名字,中哥,走神,想到,前两天,毛,不易,重点,同事,告诉,我吉恒杉,会议室,面试,一群,吻,戏,鉴定,童鞋,卧槽,一群,听,意外,收获,终归,冲进去,参观,面试,会议室,外面,刺探,情报,发现,人生,中见,奇葩,面试,面试,面试,现场,看不清,面试,试题,放大,纳尼,章子怡,有点像,杨幂,女纸,题目,扶,桌子,仔细,懂,解释一下,鬼,三十分钟,排除,一位,图里,白衣,男子,蔡,徐坤,对不起,中哥,脸盲,实在,明星,立花,美凉,大桥,未久,天海,翼,冲田杏梨三上,悠亚,无欣,作品,分辨,每次,硬,纳兰容若,那句话,人生,如初,见然,鹅,面试,个个,娱乐,圣手,认出,蔡,徐坤,几个,认出来,看着,看着,情不自禁,跪下,这位,大哥,应聘,晒,黑,大幂,幂,仔细,打听,这位,宅,男,中国,明星,是因为,中文,看剧,学会,看着,神奇,认脸,特异,走位,风骚,插不上,嘴,对话,中哥,对不起,打扰,走,错,片场,好不容易,面试,结束,老湿傅,拉,鉴吻,师,心路历程,吉恒杉,敬仰,一位,老湿傅,中哥,老湿傅,鉴吻,之路,吉湿傅,北航,计算机,视觉系,研究生,懂行,浅友,听到,这句,估计,心跳,加速,北航,计算机,视觉系,相当于,台球,界,戴斯,套套,界,杜蕾斯,手机,界,乔布斯,哲学,界,马克